<a href="https://colab.research.google.com/github/xibux-repo/kaggle/blob/master/Arvato/Arvato_Project_Workbook_zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 毕业项目：为 Arvato Financial Services 金融服务公司实现一个顾客分类报告

该项目要求你分析德国的一家邮购公司的顾客的人口统计数据，将它和一般的人口统计数据进行比较。你将使用非监督学习技术来实现顾客分类，识别出哪些人群是这家公司的基础核心用户。之后，你将把所学的知识应用到第三个数据集上，该数据集是该公司的一场邮购活动的营销对象的人口统计数据。用你搭建的模型预测哪些人更可能成为该公司的顾客。你要使用的数据由我们的合作伙伴 Bertelsmann Arvato Analytics 公司提供。这是真实场景下的数据科学任务。

如果你完成了这个纳米学位的第一学期，做过其中的非监督学习项目，那么你应该对这个项目的第一部分很熟悉了。两个数据集版本不同。这个项目中用到的数据集会包括更多的特征，而且没有预先清洗过。你也可以自由选取分析数据的方法，而不用按照既定的步骤。如果你选择完成的是这个项目，请仔细记录你的步骤和决策，因为你主要交付的成果就是一篇博客文章报告你的发现。

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#drive_path = '/content/drive/My Drive/UdacityDataScience/data/'
#workInCoLab = True

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
workInCoLab = True
drive_path = '/content/drive/My Drive/Colab Notebooks/datasets/arvato/'

In [0]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from tqdm import tqdm
tqdm.pandas()

# magic word for producing visualizations in notebook
%matplotlib inline

#!ls /content/drive/'My Drive'/'Colab Notebooks'

## 第 0 部分：了解数据

项目数据中包括四个数据文件

- `Udacity_AZDIAS_052018.csv`: 德国的一般人口统计数据；891211 人（行）x 366 个特征（列）
- `Udacity_CUSTOMERS_052018.csv`: 邮购公司顾客的人口统计数据；191652 人（行）x 369 个特征（列）
- `Udacity_MAILOUT_052018_TRAIN.csv`: 营销活动的对象的人口统计数据；42982 人（行）x 367 个特征（列）
- `Udacity_MAILOUT_052018_TEST.csv`: 营销活动的对象的人口统计数据；42833 人（行）x 366个特征（列）

人口统计数据的每一行表示是一个单独的人，也包括一些非个人特征，比如他的家庭信息、住房信息以及周边环境信息。使用前两个数据文件中的信息来发现顾客（"CUSTOMERS"）和一般人（"AZDIAS"）在何种程度上相同和不同，然后根据你的分析对其余两个数据文件（"MAILOUT"）进行预测，预测更可能成为该邮购公司的客户。

"CUSTOMERS" 文件包括三个额外的列（'CUSTOMER_GROUP'、’'ONLINE_PURCHASE' 和 'PRODUCT_GROUP'），提供了文件中顾客的更多维度的信息。原始的 "MAILOUT" 包括一个额外的列 "RESPONSE"，表示每个收到邮件的人是否成为了公司的顾客。对于 "TRAIN" 子数据集，该列被保留，但是在 "TEST" 子数据集中该列被删除了，它和你最后要在 Kaggle 比赛上预测的数据集中保留的列是对应的。

三个数据文件中其他的所有列都是相同的。要获得关于文件中列的更多信息，你可以参考 Workspace 中的两个 Excel 电子表格。[其一](./DIAS Information Levels - Attributes 2017.xlsx) 是一个所有属性和描述的列表，按照信息的类别进行排列。[其二](./DIAS Attributes - Values 2017.xlsx) 是一个详细的每个特征的数据值对应关系，按照字母顺序进行排列。

在下面的单元格中，我们提供了一些简单的代码，用于加载进前两个数据集。注意，这个项目中所有的 `.csv` 数据文件都是分号(`;`) 分割的，所以 [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) 中需要加入额外的参数以正确地读取数据。而且，考虑数据集的大小，加载整个数据集可能会花费一些时间。

你会注意到在数据加载的时候，会弹出一个警告（warning）信息。在你开始建模和分析之前，你需要先清洗一下数据。浏览一下数据集的结构，查看电子表格中信息了解数据的取值。决定一下要挑选哪些特征，要舍弃哪些特征，以及是否有些数据格式需要修订。我们建议创建一个做预处理的函数，因为你需要在使用数据训练模型前清洗所有数据集。

In [6]:
%%time
azdias = pd.read_csv(drive_path + 'Udacity_AZDIAS_052018.csv', sep=';', dtype=str)
customers = pd.read_csv(drive_path + 'Udacity_CUSTOMERS_052018.csv', sep=';', dtype=str)

# use column LNR as index
azdias.set_index('LNR', inplace=True)
customers.set_index('LNR', inplace=True)

CPU times: user 49.4 s, sys: 9.68 s, total: 59.1 s
Wall time: 1min 1s


In [7]:
personal = [
  'AGER_TYP',
  'ALTERSKATEGORIE_GROB',
  'ANREDE_KZ',
  'CJT_GESAMTTYP',
  'FINANZ_MINIMALIST',
  'FINANZ_SPARER',
  'FINANZ_VORSORGER',
  'FINANZ_ANLEGER',
  'FINANZ_UNAUFFAELLIGER',
  'FINANZ_HAUSBAUER',
  'FINANZTYP',
  'GEBURTSJAHR',
  'GFK_URLAUBERTYP',
  'GREEN_AVANTGARDE',
  'HEALTH_TYP',
  'LP_LEBENSPHASE_FEIN',
  'LP_LEBENSPHASE_GROB',
  'LP_FAMILIE_FEIN',
  'LP_FAMILIE_GROB',
  'LP_STATUS_GROB',
  'NATIONALITAET_KZ',
  'PRAEGENDE_JUGENDJAHRE',
  'RETOURTYP_BK_S',
  'SEMIO_SOZ',
  'SEMIO_FAM',
  'SEMIO_REL',
  'SEMIO_MAT',
  'SEMIO_VERT',
  'SEMIO_LUST',
  'SEMIO_ERL',
  'SEMIO_KULT',
  'SEMIO_RAT',
  'SEMIO_KRIT',
  'SEMIO_DOM',
  'SEMIO_KAEM',
  'SEMIO_PFLICHT',
  'SEMIO_TRADV',
  'SHOPPER_TYP',
  'SOHO_FLAG',
  'TITEL_KZ',
  'VERS_TYP',
  'ZABEOTYP'
  ]

personal_leave_out =['GEBURTSJAHR',
    'LP_LEBENSPHASE_FEIN',
    'LP_FAMILIE_FEIN', 
    'SOHO_FLAG'
]

personal_left = set(personal) - set(personal_leave_out)
type(personal_left)
print(len(personal))
print(len(personal_left))


42
38


In [8]:
azdias.head()
azdias.dtypes
azdias.columns.values
print(len(azdias.columns.values))
print(azdias.shape)

365
(891221, 365)


In [9]:
customers.head()
customers.dtypes
customers.columns.values
print(len(customers.columns.values))
print(customers.shape)
customers.columns[0:]

a = customers.select_dtypes(include='object')
a.shape

368
(191652, 368)


(191652, 368)

Only selcet personal features， get_dummies and add label column


In [8]:
azdias_personal = azdias[list(personal_left)]
print(azdias_personal.shape)
azdias_personal_dummies = pd.get_dummies(azdias_personal, prefix_sep='__')
azdias_personal_dummies['label']=1
azdias_personal_dummies.head()

(891221, 38)


,SEMIO_ERL__1,SEMIO_ERL__2,SEMIO_ERL__3,SEMIO_ERL__4,SEMIO_ERL__5,SEMIO_ERL__6,SEMIO_ERL__7,FINANZ_UNAUFFAELLIGER__1,FINANZ_UNAUFFAELLIGER__2,FINANZ_UNAUFFAELLIGER__3,FINANZ_UNAUFFAELLIGER__4,FINANZ_UNAUFFAELLIGER__5,PRAEGENDE_JUGENDJAHRE__0,PRAEGENDE_JUGENDJAHRE__1,PRAEGENDE_JUGENDJAHRE__10,PRAEGENDE_JUGENDJAHRE__11,PRAEGENDE_JUGENDJAHRE__12,PRAEGENDE_JUGENDJAHRE__13,PRAEGENDE_JUGENDJAHRE__14,PRAEGENDE_JUGENDJAHRE__15,PRAEGENDE_JUGENDJAHRE__2,PRAEGENDE_JUGENDJAHRE__3,PRAEGENDE_JUGENDJAHRE__4,PRAEGENDE_JUGENDJAHRE__5,PRAEGENDE_JUGENDJAHRE__6,PRAEGENDE_JUGENDJAHRE__7,PRAEGENDE_JUGENDJAHRE__8,PRAEGENDE_JUGENDJAHRE__9,SEMIO_TRADV__1,SEMIO_TRADV__2,SEMIO_TRADV__3,SEMIO_TRADV__4,SEMIO_TRADV__5,SEMIO_TRADV__6,SEMIO_TRADV__7,SEMIO_KAEM__1,SEMIO_KAEM__2,SEMIO_KAEM__3,SEMIO_KAEM__4,SEMIO_KAEM__5,...,FINANZ_MINIMALIST__1,FINANZ_MINIMALIST__2,FINANZ_MINIMALIST__3,FINANZ_MINIMALIST__4,FINANZ_MINIMALIST__5,CJT_GESAMTTYP__1,CJT_GESAMTTYP__2,CJT_GESAMTTYP__3,CJT_GESAMTTYP__4,CJT_GESAMTTYP__5,CJT_GESAMTTYP__6,LP_FAMILIE_GROB__0,LP_FAMILIE_GROB__1,LP_FAMILIE_GROB__2,LP_FAMILIE_GROB__3,LP_FAMILIE_GROB__4,LP_FAMILIE_GROB__5,SEMIO_MAT__1,SEMIO_MAT__2,SEMIO_MAT__3,SEMIO_MAT__4,SEMIO_MAT__5,SEMIO_MAT__6,SEMIO_MAT__7,ANREDE_KZ__1,ANREDE_KZ__2,SEMIO_REL__1,SEMIO_REL__2,SEMIO_REL__3,SEMIO_REL__4,SEMIO_REL__5,SEMIO_REL__6,SEMIO_REL__7,TITEL_KZ__0,TITEL_KZ__1,TITEL_KZ__2,TITEL_KZ__3,TITEL_KZ__4,TITEL_KZ__5,label
LNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
910215,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
910220,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1
910225,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1
910226,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1
910241,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


The same thing again for customers

In [9]:
customers_personal = customers[list(personal_left)]
print(customers_personal.shape)
customers_personal_dummies = pd.get_dummies(customers_personal, prefix_sep='__')
customers_personal_dummies['label']=0
customers_personal_dummies.head()

(191652, 38)


,SEMIO_ERL__1,SEMIO_ERL__2,SEMIO_ERL__3,SEMIO_ERL__4,SEMIO_ERL__5,SEMIO_ERL__6,SEMIO_ERL__7,FINANZ_UNAUFFAELLIGER__1,FINANZ_UNAUFFAELLIGER__2,FINANZ_UNAUFFAELLIGER__3,FINANZ_UNAUFFAELLIGER__4,FINANZ_UNAUFFAELLIGER__5,PRAEGENDE_JUGENDJAHRE__0,PRAEGENDE_JUGENDJAHRE__1,PRAEGENDE_JUGENDJAHRE__10,PRAEGENDE_JUGENDJAHRE__11,PRAEGENDE_JUGENDJAHRE__12,PRAEGENDE_JUGENDJAHRE__13,PRAEGENDE_JUGENDJAHRE__14,PRAEGENDE_JUGENDJAHRE__15,PRAEGENDE_JUGENDJAHRE__2,PRAEGENDE_JUGENDJAHRE__3,PRAEGENDE_JUGENDJAHRE__4,PRAEGENDE_JUGENDJAHRE__5,PRAEGENDE_JUGENDJAHRE__6,PRAEGENDE_JUGENDJAHRE__7,PRAEGENDE_JUGENDJAHRE__8,PRAEGENDE_JUGENDJAHRE__9,SEMIO_TRADV__1,SEMIO_TRADV__2,SEMIO_TRADV__3,SEMIO_TRADV__4,SEMIO_TRADV__5,SEMIO_TRADV__6,SEMIO_TRADV__7,SEMIO_KAEM__1,SEMIO_KAEM__2,SEMIO_KAEM__3,SEMIO_KAEM__4,SEMIO_KAEM__5,...,LP_LEBENSPHASE_GROB__9,FINANZ_MINIMALIST__1,FINANZ_MINIMALIST__2,FINANZ_MINIMALIST__3,FINANZ_MINIMALIST__4,FINANZ_MINIMALIST__5,CJT_GESAMTTYP__1,CJT_GESAMTTYP__2,CJT_GESAMTTYP__3,CJT_GESAMTTYP__4,CJT_GESAMTTYP__5,CJT_GESAMTTYP__6,LP_FAMILIE_GROB__0,LP_FAMILIE_GROB__1,LP_FAMILIE_GROB__2,LP_FAMILIE_GROB__3,LP_FAMILIE_GROB__4,LP_FAMILIE_GROB__5,SEMIO_MAT__1,SEMIO_MAT__2,SEMIO_MAT__3,SEMIO_MAT__4,SEMIO_MAT__5,SEMIO_MAT__6,SEMIO_MAT__7,ANREDE_KZ__1,ANREDE_KZ__2,SEMIO_REL__1,SEMIO_REL__2,SEMIO_REL__3,SEMIO_REL__4,SEMIO_REL__5,SEMIO_REL__6,SEMIO_REL__7,TITEL_KZ__0,TITEL_KZ__1,TITEL_KZ__3,TITEL_KZ__4,TITEL_KZ__5,label
LNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9626,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
9628,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
143872,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0
143873,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
143874,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0


Union azdias and customers, 'TITEL_KZ__2' is not in customers, so we drop it. 


In [10]:
union_all = pd.concat([customers_personal_dummies, azdias_personal_dummies])
union_all.drop(columns=['TITEL_KZ__2'], inplace=True)
print(union_all.shape)
union_all.head() # this is the training data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(1082873, 239)


,AGER_TYP__-1,AGER_TYP__0,AGER_TYP__1,AGER_TYP__2,AGER_TYP__3,ALTERSKATEGORIE_GROB__1,ALTERSKATEGORIE_GROB__2,ALTERSKATEGORIE_GROB__3,ALTERSKATEGORIE_GROB__4,ALTERSKATEGORIE_GROB__9,ANREDE_KZ__1,ANREDE_KZ__2,CJT_GESAMTTYP__1,CJT_GESAMTTYP__2,CJT_GESAMTTYP__3,CJT_GESAMTTYP__4,CJT_GESAMTTYP__5,CJT_GESAMTTYP__6,FINANZTYP__1,FINANZTYP__2,FINANZTYP__3,FINANZTYP__4,FINANZTYP__5,FINANZTYP__6,FINANZ_ANLEGER__1,FINANZ_ANLEGER__2,FINANZ_ANLEGER__3,FINANZ_ANLEGER__4,FINANZ_ANLEGER__5,FINANZ_HAUSBAUER__1,FINANZ_HAUSBAUER__2,FINANZ_HAUSBAUER__3,FINANZ_HAUSBAUER__4,FINANZ_HAUSBAUER__5,FINANZ_MINIMALIST__1,FINANZ_MINIMALIST__2,FINANZ_MINIMALIST__3,FINANZ_MINIMALIST__4,FINANZ_MINIMALIST__5,FINANZ_SPARER__1,...,SEMIO_SOZ__2,SEMIO_SOZ__3,SEMIO_SOZ__4,SEMIO_SOZ__5,SEMIO_SOZ__6,SEMIO_SOZ__7,SEMIO_TRADV__1,SEMIO_TRADV__2,SEMIO_TRADV__3,SEMIO_TRADV__4,SEMIO_TRADV__5,SEMIO_TRADV__6,SEMIO_TRADV__7,SEMIO_VERT__1,SEMIO_VERT__2,SEMIO_VERT__3,SEMIO_VERT__4,SEMIO_VERT__5,SEMIO_VERT__6,SEMIO_VERT__7,SHOPPER_TYP__-1,SHOPPER_TYP__0,SHOPPER_TYP__1,SHOPPER_TYP__2,SHOPPER_TYP__3,TITEL_KZ__0,TITEL_KZ__1,TITEL_KZ__3,TITEL_KZ__4,TITEL_KZ__5,VERS_TYP__-1,VERS_TYP__1,VERS_TYP__2,ZABEOTYP__1,ZABEOTYP__2,ZABEOTYP__3,ZABEOTYP__4,ZABEOTYP__5,ZABEOTYP__6,label
LNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9626,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
9628,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
143872,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
143873,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
143874,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0


Clustering

In [11]:
from sklearn.cluster import KMeans
X = union_all.drop(columns=['label'])
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [12]:
true_labels = union_all['label']
print(true_labels.head())
pred_labels = kmeans.labels_
print(pred_labels[0:5])
acc = sum(true_labels == pred_labels)/len(union_all)
print(acc)

from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(true_labels, pred_labels) 

LNR
9626      0
9628      0
143872    0
143873    0
143874    0
Name: label, dtype: int64
[0 0 0 0 0]
0.20786740458022315


0.1279537661748935

In [0]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=3, method='barnes_hut').fit_transform(X)

In [0]:
from mpl_toolkits.mplot3d import Axes3D

def tsne_mapping(trData, method, perplexity):
    
    dataShape = trData.shape
    print str(dataShape[0]) + " data points in " + str(dataShape[1]) + " dimensions"
    
    if perplexity <= 0:
        perplexity = np.floor( np.log( dataShape[0] )**2 )
        print "Perplexity = " + str(perplexity)
    else:
        print "Set perlexity to " + str(perplexity)

    model = TSNE(n_components=3, random_state=0, perplexity=perplexity, method=method) # method: barnes_hut, 
    
#     np.set_printoptions(suppress=True)
    tsne_output = model.fit_transform(trData) 
    #print tsne_output
    
    return tsne_output


def plot_3d_with_labels_df(df, name_of_column_as_label):
    
    fig = plt.figure(figsize=(15,10))
    ax = fig.gca(projection='3d') 
    
    uniq_labels = df[name_of_column_as_label].unique()
    
    colors = get_distinguishable_colors(len(uniq_labels))
    markers = get_distinguish_markers(len(uniq_labels))
    
    for i in range(len(uniq_labels)):
        d2p = df[df[name_of_column_as_label]==uniq_labels[i]]
        print d2p
        ax.plot(d2p[:,0], d2p[:,1], d2p[:,2], marker=markers[i], markersize=5, alpha=0.5, label=uniq_labels[i], color=colors[i])
        
    ax.legend(loc='best')

In [1]:
X_embedded

NameError: ignored


## 第0部分：清洗数据 cleaning Data


### 警告形象对应的数据问题

首先我们看看警告所提出的问题，18和19列里到底有什么样的数据问题？

In [46]:
print('column 18 label is', azdias.columns[18])
print('column 19 label is', azdias.columns[19])
print(azdias[azdias.columns[18]].unique())
print(azdias[azdias.columns[19]].unique())

column 18 label is CAMEO_INTL_2015
column 19 label is CJT_GESAMTTYP
[nan '51' '24' '12' '43' '54' '22' '14' '13' '15' '33' '41' '34' '55' '25'
 '23' '31' '52' '35' '45' '44' '32' 'XX']
['2' '5' '3' '4' '1' '6' nan]


In [0]:
def findObjectAttributs(dataframe):
  '''
  find which column in dataframe has object as dtype. 
  Args:
    dataframe {DataFrame} -- it could be customer or azdias
  Returns:
    {set} -- a set of column names, those the dtypes of column is value type 
      object
  '''
  #object_columns = set()
  for attr in dataframe.columns[1:]: 
    attr_unique_values = dataframe[attr].unique()
    if dataframe[attr].dtypes == "object": 
      #object_columns.add(attr)
      print(f'{attr} has value {attr_unique_values}')
  #return object_columns

print('== findObjectAttributs(azdias) ==')
findObjectAttributs(azdias)

print('== findObjectAttributs(customers) ==')
findObjectAttributs(customers)

居于上面的方向我们发现了这里有 X 和 XX 的数据，对应于数据描述./DIAS Attributes - Values 2017.xlsx这些并不市有效的值。同时还存在string和number共同存在这些列中。我们将先对这两列作相应的处理。同时我们发现作为需要的值是可以处理为-1，表示unknown。

In [10]:
def cleanupCameoDeu2015(dataframe):
  dataframe['CAMEO_DEU_2015'] = dataframe['CAMEO_DEU_2015'].replace('XX', np.nan)
  print(f'after cleanup column CAMEO_DEU_2015 has values: {dataframe["CAMEO_DEU_2015"].unique()}\n')

  dataframe['CAMEO_DEUG_2015'] = dataframe['CAMEO_DEUG_2015']\
                                  .replace('X', np.nan)\
                                  .map(lambda x: str(x)[0])\
                                  .map(lambda x: np.nan if x in ['n'] else x)
  print(f'after cleanup column CAMEO_DEUG_2015 has values: {dataframe["CAMEO_DEUG_2015"].unique()}\n')

  dataframe['CAMEO_INTL_2015'] = dataframe['CAMEO_INTL_2015']\
                              .replace('XX', np.nan)\
                              .map(lambda x: str(x)[0:1] if str(x)[-2:]=='.0' else x)\
                              .map(lambda y: np.nan if y == 'na' else y)
  print(f'after cleanup column CAMEO_INTL_2015 has values: {dataframe["CAMEO_INTL_2015"].unique()}\n')


print("== cleanupCameoDeu2015 in azdias ==")
cleanupCameoDeu2015(azdias)

print("== cleanupCameoDeu2015 in customers ==")
cleanupCameoDeu2015(customers)

== cleanupCameoDeu2015 in azdias ==
after cleanup column CAMEO_DEU_2015 has values: [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C']

after cleanup column CAMEO_DEUG_2015 has values: [nan '8' '4' '2' '6' '1' '9' '5' '7' '3']

after cleanup column CAMEO_INTL_2015 has values: [nan '51' '24' '12' '43' '54' '22' '14' '13' '15' '33' '41' '34' '55' '25'
 '23' '31' '52' '35' '45' '44' '32']

== cleanupCameoDeu2015 in customers ==
after cleanup column CAMEO_DEU_2015 has values: ['1A' nan '5D' '4C' '7B' '3B' '1D' '9E' '2D' '4A' '6B' '9D' '8B' '5C' '9C'
 '4E' '6C' '8C' '8A' '5B' '9B' '3D' '2A' '3C' '5F' '7A' '1E' '2C' '7C'
 '5A' '2B' '6D' '7E' '5E' '6E' '3A' '9A' '4B' '1C' '1B' '6A' '8D' '7D'
 '6F' '4D']

after cleanup column CAMEO_DEUG_2015 has values: ['1' nan '5' '4' '7' '3' '9' '2' '6' '8']

after cleanup column CA

整个清理数据的目标就是，我们利用pandas的get_dummies方法可以将这个attribute对应成以键值的特征矩阵，后面可以替换对应的attribute列。比如列CAMEO_INTL_2015将被下面的dummies替换。下面是一个例子，dummis将会是一个以LNR为索引，具体attribute__value为列的特征矩阵。

In [17]:
dummies = pd.get_dummies(azdias, columns=['CAMEO_DEU_2015', 'CAMEO_INTL_2015'], prefix_sep='__')
dummies

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEUG_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,...,CAMEO_DEU_2015__6B,CAMEO_DEU_2015__6C,CAMEO_DEU_2015__6D,CAMEO_DEU_2015__6E,CAMEO_DEU_2015__6F,CAMEO_DEU_2015__7A,CAMEO_DEU_2015__7B,CAMEO_DEU_2015__7C,CAMEO_DEU_2015__7D,CAMEO_DEU_2015__7E,CAMEO_DEU_2015__8A,CAMEO_DEU_2015__8B,CAMEO_DEU_2015__8C,CAMEO_DEU_2015__8D,CAMEO_DEU_2015__9A,CAMEO_DEU_2015__9B,CAMEO_DEU_2015__9C,CAMEO_DEU_2015__9D,CAMEO_DEU_2015__9E,CAMEO_INTL_2015__12,CAMEO_INTL_2015__13,CAMEO_INTL_2015__14,CAMEO_INTL_2015__15,CAMEO_INTL_2015__22,CAMEO_INTL_2015__23,CAMEO_INTL_2015__24,CAMEO_INTL_2015__25,CAMEO_INTL_2015__31,CAMEO_INTL_2015__32,CAMEO_INTL_2015__33,CAMEO_INTL_2015__34,CAMEO_INTL_2015__35,CAMEO_INTL_2015__41,CAMEO_INTL_2015__43,CAMEO_INTL_2015__44,CAMEO_INTL_2015__45,CAMEO_INTL_2015__51,CAMEO_INTL_2015__52,CAMEO_INTL_2015__54,CAMEO_INTL_2015__55
LNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,1,1,5,5,5,5,0,0,10,0,0,0,10,10,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
910220,-1,9,0,NaN,NaN,NaN,NaN,21,11,0,0,2,12,0,3,6,8,5,1,5,5,2,3,1,1,0,0,10,0,0,0,10,10,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
910225,-1,9,17,NaN,NaN,NaN,NaN,17,10,0,0,1,7,0,3,2,4,3,2,4,4,1,3,2,2,0,0,10,0,0,0,10,10,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
910226,2,1,13,NaN,NaN,NaN,NaN,13,1,0,0,0,2,0,2,4,2,2,3,2,2,4,4,5,3,0,0,10,0,0,0,10,10,0,0,0,0,0,0,6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
910241,-1,1,20,NaN,NaN,NaN,NaN,14,3,0,0,4,3,0,4,2,6,5,3,3,3,3,4,3,3,3,5,5,1,2,0,10,5,10,6,6,1,6,0,6,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825761,-1,5,17,NaN,NaN,NaN,NaN,17,15,0,0,1,11,0,NaN,6,7,5,2,5,4,3,3,1,2,0,0,10,0,0,0,10,10,0,0,0,5,0,0,4,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
825771,-1,9,16,NaN,NaN,NaN,NaN,16,11,0,0,1,11,0,4,7,9,4,3,5,5,1,3,1,3,0,0,10,0,0,0,10,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
825772,-1,1,17,NaN,NaN,NaN,NaN,17,3,0,0,0,3,0,2,5,4,4,1,5,4,3,1,1,2,0,0,9,6,0,0,10,9,0,0,5,6,0,0,6,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
pd.get_dummies(azdias, prefix_sep='__')

In [0]:
del dummies

### Attributes和Values 

Arvato 提供了对应的Meta数据集，Attributes是所有像个的描述。Values包含了每个Attribute可能的数据值，以及其对应的意义Meaning。这里我们把他读取出来，整理成一些相应需要的变量，为后面的数据清洗做准备。特别是Unknown数据，我们可以利用描述来找到对应的值，准备好清洗掉它们。

In [53]:
attributes_df = pd.read_excel(drive_path + 'DIAS Information Levels - Attributes 2017.xlsx', index_col=None, header=1)
values_df = pd.read_excel(drive_path + 'DIAS Attributes - Values 2017.xlsx', index_col=None, header=1)

del attributes_df['Unnamed: 0']
del values_df['Unnamed: 0']

values_df = values_df.fillna(method='ffill', axis=0) # fill merged-cell with first value in above
values_df.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly


In [54]:
attributes_df.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [55]:
# display all possilbe meaning has 'unknown'
meaning_se = pd.Series(values_df['Meaning'].unique())
meaning_se[meaning_se.str.contains('known', flags=re.IGNORECASE, regex=True)]

0                                                unknown
10                      unknown / no main age detectable
129                                no transactions known
145                                 no transaction known
199    residental building buildings without actually...
201    mixed building without actually known househol...
202                  company building w/o known company 
203     mixed building without actually known household 
205       mixed building without actually known company 
dtype: object

In [0]:
def setUnknownValueToNan(dataframe):
  ''' 
  calculate the column names of the with meaning 'unknown' or simular meaning.
  the unknown value -1, we will do general replace, so we do ignore it.

  Args:
    None
  Returns:
    {set} of all possible attirbute columns. eg. "D19_VERSI_ANZ_12__0" 
  '''
  unknown = ['unknown', 
            'unknown / no main age detectable',
            'no transactions known',
            'no transaction known']

  attribute_unknown = values_df[values_df['Meaning'].isin(unknown)]
  attribute_unknown['Value'].astype(str).map(lambda st: st.split(', '))
  unknown_columns = dict()

  for index, row in attribute_unknown.iterrows():
    attribute_name = row['Attribute'].replace('_RZ', '')
    attribute_values = str(row['Value']).split(', ')
    for unknown_val in attribute_values:
      if unknown_val == '-1':
        continue
      #unknown_columns.add(f'{attribute_name}__{unknown_val}')
      unknown_columns[attribute_name] = unknown_val
  
  for (key, value) in unknown_columns.items():
    if key in azdias.columns:
        dataframe[key].replace(value, np.nan, inplace=True)
    else:
        print(f'Can not find {key} column，just skip')

In [57]:
setUnknownValueToNan(azdias)

Can not find D19_BUCH column，just skip
Can not find GEOSCORE_KLS7 column，just skip
Can not find HAUSHALTSSTRUKTUR column，just skip
Can not find WACHSTUMSGEBIET_NB column，just skip


attributes 和 values定义和实际数据中使用的差距

In [59]:
azdias_attributes = azdias.columns
customer_attributes = customers.columns
attributes_in_valuesDf = values_df['Attribute'].unique()
attributes_in_attributesDf = attributes_df['Attribute'].unique()

print(f'there are {azdias_attributes.size} in azdias')
print(f'there are {customer_attributes.size} in customers')
print(f'there are {attributes_in_valuesDf.shape[0]} in values_df')
print(f'there are {attributes_in_attributesDf.shape[0]} in attributes_df')

there are 365 in azdias
there are 368 in customers
there are 314 in values_df
there are 313 in attributes_df


为什么在azdias和customer的特征列数量和描述数据values和attributs不对应呢？这里我们进一步通过显示数据来做分析。

In [61]:
attributes_without_meta = set()
ind = 0
for attr in azdias_attributes:
  ind += 1
  if attr not in attributes_in_valuesDf:
    attributes_without_meta.add(attr)
    print(f'Attribute No.{ind} {attr} is not in attributes_df, but in azdias values are: {azdias[attr].unique()}')

Attribute No.2 AKT_DAT_KL is not in attributes_df, but in azdias values are: [nan '9' '1' '5' '8' '7' '6' '4' '3' '2']
Attribute No.4 ALTER_KIND1 is not in attributes_df, but in azdias values are: [nan '17' '10' '18' '13' '16' '11' '6' '8' '9' '15' '14' '7' '12' '4' '3'
 '5' '2']
Attribute No.5 ALTER_KIND2 is not in attributes_df, but in azdias values are: [nan '13' '8' '12' '10' '7' '16' '15' '14' '17' '5' '9' '18' '11' '6' '4'
 '3' '2']
Attribute No.6 ALTER_KIND3 is not in attributes_df, but in azdias values are: [nan '10' '18' '17' '16' '8' '15' '9' '12' '13' '14' '11' '7' '5' '6' '4']
Attribute No.7 ALTER_KIND4 is not in attributes_df, but in azdias values are: [nan '10' '9' '16' '14' '13' '11' '18' '17' '15' '8' '12' '7']
Attribute No.8 ALTERSKATEGORIE_FEIN is not in attributes_df, but in azdias values are: [nan '21' '17' '13' '14' '10' '16' '20' '11' '19' '15' '18' '9' '22' '12'
 '0' '8' '7' '23' '4' '24' '6' '3' '2' '5' '25' '1']
Attribute No.11 ANZ_KINDER is not in attributes_d

通过观察我们发现，D19_LETZTER_KAUF_BRANCHE的值刚好对应了其他D19的列，数据看上去有一定的重复。CJT_KATALOGNUTZER也是类似情况，被其他CJT列所重复。ANZ_STATISTISCHE_HAUSHALTE，EXTSEL992有大量的数值，但是我们这里缺乏具体的meta数据，这里我们决定不再保留。EINGEZOGENAM_HH_JAHR。GEBURTSJAHR是出身年份，我们还有其他的列含有相关年龄的列ALTER_HH所以我们也决定忽略。

In [65]:
azdias_shapeBefore = azdias.shape[1]
customers_shapeBefore = customers.shape[1]
columns_to_drop = {'D19_LETZTER_KAUF_BRANCHE', 
                   'CJT_KATALOGNUTZER', 
                   'EINGEZOGENAM_HH_JAHR', 
                   'ANZ_STATISTISCHE_HAUSHALTE',
                   'ANZ_HAUSHALTE_AKTIV',
                   'VERDICHTUNGSRAUM', 
                   'EXTSEL992',
                   'GEBURTSJAHR',
                   'ALTER_KIND1',
                   'ALTER_KIND2',
                   'ALTER_KIND3',
                   'ALTER_KIND4'}

azdias.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)
customers.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)

columns_to_drop.clear()

print(f'before azdias drop {azdias_shapeBefore}, and after {azdias.shape}')
print(f'before customers drop {customers_shapeBefore}, and customers {customers.shape}')

before azdias drop 353, and after (891221, 353)
before customers drop 356, and customers (191652, 356)


In [0]:
# extract special columns from customers
customer_special_columns = ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']
customer_special_df = customers[customer_special_columns]

print(f'columns {customer_special_columns} will be delete from customers')
customers.drop(columns=customer_special_columns, axis=1, errors='ignore', inplace=True)

customer_special_df.head()

EINGEFUEGT_AM是数据添加的时间，一共有5163个时间点，我们只采纳的年份作为特这。将其替换为EINGEFUEGT_AM将会只是数据输入的年份。

In [0]:
def pickYearValue(dataframe, attributeName):
  '''
  use only the year of timestamp as value for the attribute
  Args:
    dataframe {DataFrame} -- customers or azdias
    attributeName {string} -- attribute name, which hast timestamp value 
      like 1992-02-10 00:00:00
  Returns:
    None
  '''
  attr_values = dataframe[attributeName].unique()
  print(f'Attribute {attributeName} has {attr_values.shape[0]} values')
  print('Before change:\n', dataframe[attributeName].head())
  dataframe[attributeName] = dataframe[attributeName]\
                            .map(lambda x: str(x)[:4] if x != np.nan else x)\
                            .map(lambda x: np.nan if x=='nan' else x)
  print(f'After change:\n', dataframe[attributeName].head())
  print('We replaced dataframe[attributeName] with only the year values:',
        dataframe[attributeName].unique())


print("== pickYearValue(customers, 'EINGEFUEGT_AM') ==")
pickYearValue(customers, 'EINGEFUEGT_AM')

print("== pickYearValue(azdias, 'EINGEFUEGT_AM') ==")
pickYearValue(azdias, 'EINGEFUEGT_AM')

#### 在FEIN和GROB数据中做选择

在属性描述中我们看到，有不少属性我们同时拥有细化（FEIN）和粗略（GROB）的数据特征。这里在试验初期我们决定采用粗略的特征。

In [0]:
print(azdias['LP_FAMILIE_FEIN'].unique())
print(azdias['LP_FAMILIE_GROB'].unique())

In [0]:
for attr in attributes_in_attributesDf:
  if attr.endswith('_FEIN'):
    columns_to_drop.add(attr)

print(f'== delete {columns_to_drop} from azdias ==')
azdias.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)

print(f'== delete {columns_to_drop} from customers ==')
customers.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)
columns_to_drop.clear()

In [0]:
del values_df
del attributes_df

In [0]:
def findObjectAttributs(dataframe):
  '''
  find which column in dataframe has object as dtype. 
  Args:
    dataframe {DataFrame} -- it could be customer or azdias
  Returns:
    {set} -- a set of column names, those the dtypes of column is value type 
      object
  '''
  object_columns = set()
  for (columnName, columnData) in dataframe.iteritems():
    attr_unique_values = columnData.unique()
    if columnData.dtypes == "object": 
      object_columns.add(columnName)
      print(f'{columnName} has value {attr_unique_values}')
  return object_columns

print('== findObjectAttributs(azdias) ==')
object_dtype_columns = findObjectAttributs(azdias)
print('== findObjectAttributs(customers) ==')
findObjectAttributs(customers)

In [0]:
def findAttributesWithMuchValues(dataframe, valueSizeLimit = 10):
  '''
  find attributes/columns in dataframe, they have an oversize of the values. 

  Args:
    dataframe {DataFrame} -- it could be customers and azdias
    valueSizeLimit {int} -- the limitation of value size you want to check, 
      default 10
  Returns:
    {set} -- all attributs, in dataframe has more than 10 values
  '''
  value_oversize_columns = set()
  for (columnName, columnData) in dataframe.iteritems():
    attr_unique_values = columnData.unique()
    if attr_unique_values.size >= valueSizeLimit: 
      value_oversize_columns.add(columnName)
      print(f'{columnName} has value {attr_unique_values}')
  #return value_oversize_columns

print('== findAttributesWithMuchValues(azdias) ==')
findAttributesWithMuchValues(azdias)

print('== findAttributesWithMuchValues(customers) ==')
findAttributesWithMuchValues(customers)

In [0]:
azdias.fillna(88, inplace=True)
azdias

In [0]:
azdias.replace(-1, 88, inplace=True)

In [0]:
firstPart=azdias[:10]
firstPart

In [0]:
firstPart = azdias[:10]
firstPart = firstPart.replace(-1, 88).astype(np.int32).astype(str).replace('88', np.nan)
firstPart

In [0]:

dummies = pd.get_dummies(firstPart, prefix_sep='__', sparse=True)
dummies


In [0]:
del dummies

In [0]:
dummies = pd.get_dummies(azdias, columns=object_dtype_columns, prefix_sep='__', sparse=True, dtype=np.int16)
azdias.drop(columns=object_dtype_columns, axis=1, errors='ignore', inplace=True)

In [0]:
print(dummies.columns.values)

In [0]:
azdias.columns.values

In [0]:
dummnies.columns.values

In [0]:
azdias.fillna('88').astype(np.int16).astype(str).applymap(lambda x: np.nan if x == '88' else x)

In [0]:
azdias.drop(columns=object_dtype_columns, axis=1, inplace=True, errors='ignore')

def wrapValuesToIntAndKeepNaN(dataFrame):
    for col in azdias.columns.values:
        azdias[col].fillna(88).astype(np.int16).astype(str).apply(lambda x: np.nan if x == '88' else x)

In [0]:
dummies = pd.get_dummies(azdias, prefix_sep='__', sparse=True)
azdias.drop(columns=object_columns, inplace=True)
dummnies.columns.values

In [0]:
dummies = pd.get_dummies(azdias['MIN_GEBAEUDEJAHR'], prefix='MIN_GEBAEUDEJAHR', prefix_sep="__")
dummies

## 第1部分：顾客分类报告

项目报告的主体部分应该就是这部分。在这个部分，你应该使用非监督学习技术来刻画公司已有顾客和德国一般人群的人口统计数据的关系。这部分做完后，你应该能够描述一般人群中的哪一类人更可能是邮购公司的主要核心顾客，哪些人则很可能不是。

## 第2部分：监督学习模型

你现在应该已经发现哪部分人更可能成为邮购公司的顾客了，是时候搭建一个预测模型了。"MAILOUT"数据文件的的每一行表示一个邮购活动的潜在顾客。理想情况下我们应该能够使用每个人的人口统计数据来决定是否该把他作为该活动的营销对象。

"MAILOUT" 数据被分成了两个大致相等的部分，每部分大概有 43 000 行数据。在这部分，你可以用"TRAIN"部分来检验你的模型，该数据集包括一列"RESPONSE"，该列表示该对象是否参加了该公司的邮购活动。在下一部分，你需要在"TEST"数据集上做出预测，该数据集中"RESPONSE" 列也被保留了。

In [0]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [0]:
mailout_train = pd.read_csv(drive_path+'Udacity_MAILOUT_052018_TRAIN.csv', sep=';')


In [0]:
mailout_train.head

## Part 3:Kaggle比赛

你已经搭建了一个用于预测人们有多大程度上会回应邮购活动的模型，是时候到Kaggle上检验一下这个模型了。如果你点击这个 [链接](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140)，你会进入到比赛界面（如果你已经有一个Kaggle账户的话）如果你表现突出的话，你将有机会收到Arvato或Bertelsmann的人力资源管理的经理的面试邀约！

你比赛用提交的文件格式为CSV，该文件含2列。第一列是"LNR"，是"TEST"部分每个顾客的ID。第二列是"RESPONSE"表示此人有多大程度上会参加该活动，可以是某种度量，不一定是概率。你应该在第2部分已经发现了，该数据集存在一个巨大的输出类不平衡的问题，也就是说大部分人都不会参加该邮购活动。因此，预测目标人群的分类并使用准确率来衡量不是一个合适的性能评估方法。相反地，该项竞赛使用AUC衡量模型的性能。"RESPONSE"列的绝对值并不重要：仅仅表示高的取值可能吸引到更多的实际参与者，即ROC曲线的前端曲线比较平缓。

In [0]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

```python

```